---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
def graph_identification():
    
    return ['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [4]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [5]:
def salary_predictions():
    
    from sklearn import svm
    from sklearn.ensemble import GradientBoostingClassifier
    
    #iner function 
    def has_management_salary(node):
        management_salary = node[1]['ManagementSalary']
        
        if management_salary == 1:
            return 1
        
        elif management_salary == 0:
            return 0
        
        else:
            return None
        
        
    #now analyze the data
    df = pd.DataFrame(index=G.nodes())
    
    #add feature values
    df['Degree'] = pd.Series(nx.degree(G))
    df['DegreeCentrality'] = pd.Series(nx.degree_centrality(G))
    df['BetweennessCentrality'] = pd.Series(nx.betweenness_centrality(G))
    df['PageRank'] = pd.Series(nx.pagerank(G))
    
    #target value
    management_series = [has_management_salary(node) for node in G.nodes(data=True)]
    df['ManagementSalary'] = pd.Series(management_series)
    
    #create test and train dataframes
    df_train = df[df['ManagementSalary'].notnull()]
    df_test = df[~df['ManagementSalary'].notnull()]
    
    #create test and train datasets
    X_train = df_train[['Degree', 'DegreeCentrality', 'BetweennessCentrality', 'PageRank']]
    Y_train = df_train['ManagementSalary']
    
    X_test = df_test[['Degree', 'DegreeCentrality', 'BetweennessCentrality', 'PageRank']]
    
    #create classifier -- it given ~0.77AUC score
    #clf = svm.SVC(probability=True, gamma=0.001, C=100.)
    #clf.fit(X_train, Y_train)
    
    #Y_test = clf.predict(X_test)
    #test_prob = clf.predict_proba(X_test)[:, 1]
    
    #second try -- ~0.85AUC score, still below 0.88 
    clf = GradientBoostingClassifier()
    clf.fit(X_train, Y_train)
    
    test_prob = clf.predict_proba(X_test)[:, 1]
    
    
    return pd.Series(test_prob, X_test.index)

salary_predictions()

1       0.102094
2       0.987008
5       0.987008
8       0.466218
14      0.062634
18      0.135630
27      0.121806
30      0.076100
31      0.823244
34      0.194173
37      0.481569
40      0.287278
45      0.039529
54      0.611746
55      0.051729
60      0.628368
62      0.987008
65      0.987008
77      0.016533
79      0.259519
97      0.016533
101     0.064603
103     0.197680
108     0.092978
113     0.073951
122     0.098255
141     0.030676
142     0.987008
144     0.017092
145     0.113512
          ...   
913     0.014859
914     0.020900
915     0.010820
918     0.098255
923     0.015626
926     0.017092
931     0.126648
934     0.015119
939     0.015119
944     0.010820
945     0.015119
947     0.100143
950     0.058913
951     0.017092
953     0.013510
959     0.010820
962     0.015119
963     0.127889
968     0.025064
969     0.051864
974     0.015119
984     0.029202
987     0.017092
989     0.025064
991     0.025064
992     0.025064
994     0.010820
996     0.0108

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [6]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [17]:
def new_connections_predictions():
    
    from sklearn.ensemble import GradientBoostingClassifier
    
    #add features
    future_connections['Common Neighbours'] = [len(list(nx.common_neighbors(G, x[0], x[1]))) for x in future_connections.index]
    future_connections['Jaccard Coefficient'] = [x[2] for x in nx.jaccard_coefficient(G, future_connections.index)]
    future_connections['Resource Allocation'] = [x[2] for x in nx.resource_allocation_index(G, future_connections.index)]
    future_connections['Preffered Attachment'] = [list(nx.preferential_attachment(G, [node_pair]))[0][2]
                                             for node_pair in future_connections.index]
    
    #train&test data
    #create test and train dataframes
    future_connections_train = future_connections[future_connections['Future Connection'].notnull()]
    future_connections_test = future_connections[~future_connections['Future Connection'].notnull()]
    
    #create test and train datasets
    X_train = future_connections_train[['Common Neighbours', 'Jaccard Coefficient', 'Resource Allocation', 'Preffered Attachment']]
    Y_train = future_connections_train['Future Connection']
    
    X_test = future_connections_test[['Common Neighbours', 'Jaccard Coefficient', 'Resource Allocation', 'Preffered Attachment']]
 
    clf = GradientBoostingClassifier()
    clf.fit(X_train, Y_train)
    pred = clf.predict_proba(X_test)[:,1]
    return pd.Series(pred, index=X_test.index)
    
    #return ':C'

new_connections_predictions()

(107, 348)    0.029567
(542, 751)    0.013001
(20, 426)     0.562763
(50, 989)     0.013001
(942, 986)    0.013001
(324, 857)    0.013001
(13, 710)     0.118918
(19, 271)     0.138913
(319, 878)    0.013001
(659, 707)    0.013001
(49, 843)     0.013001
(208, 893)    0.013001
(377, 469)    0.013475
(405, 999)    0.019043
(129, 740)    0.015585
(292, 618)    0.049497
(239, 689)    0.013001
(359, 373)    0.013475
(53, 523)     0.031658
(276, 984)    0.013001
(202, 997)    0.013001
(604, 619)    0.092708
(270, 911)    0.013001
(261, 481)    0.059519
(200, 450)    0.904397
(213, 634)    0.013001
(644, 735)    0.154224
(346, 553)    0.013001
(521, 738)    0.013001
(422, 953)    0.018690
                ...   
(672, 848)    0.013001
(28, 127)     0.970584
(202, 661)    0.013001
(54, 195)     0.997258
(295, 864)    0.013001
(814, 936)    0.013001
(839, 874)    0.013001
(139, 843)    0.013001
(461, 544)    0.013173
(68, 487)     0.013173
(622, 932)    0.013001
(504, 936)    0.016074
(479, 528) 